In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import NearMiss
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import RepeatedEditedNearestNeighbours
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from sklearn.model_selection import KFold
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv('creditcard.csv')

In [3]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
data['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [5]:
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:,0:30], 
                                                    data.iloc[:,30], 
                                                    test_size=0.2, 
                                                    random_state=41)

# Iteration 1: Logistic regression with weighted classes

In [6]:
clf_base = LogisticRegression()
parameters = {'C': [0.01],
              'penalty': ['l1', 'l2'],
              'class_weight': ['balanced']
             }
k_fold = KFold(n_splits = 10, shuffle = False, random_state = 42)
clf = GridSearchCV(clf_base, param_grid = parameters, cv = k_fold, n_jobs = 2, scoring = 'f1_macro')
clf.fit(X_train, y_train)
prediction = clf.predict(X_test)

In [7]:
print (classification_report(y_test, prediction))

             precision    recall  f1-score   support

          0       1.00      0.98      0.99     56868
          1       0.07      0.89      0.12        94

avg / total       1.00      0.98      0.99     56962



# Iteration 2: Random undersampling

In [8]:
rus = RandomUnderSampler(ratio = 'majority', random_state=42)
X_res, y_res = rus.fit_sample(X_train, y_train)

clf_base = RandomForestClassifier(random_state = 42)
parameters = {'n_estimators': [32],
             'criterion': ['gini'],
             'max_depth': [7],
             'class_weight': ['balanced']
             }
k_fold = KFold(n_splits = 10, shuffle = True, random_state = 42)
clf = GridSearchCV(clf_base, param_grid = parameters, cv = k_fold, n_jobs = 3, scoring = 'f1_macro')
clf.fit(X_res, y_res)
prediction = clf.predict(X_test)

In [9]:
print (classification_report(y_test, prediction))

             precision    recall  f1-score   support

          0       1.00      0.98      0.99     56868
          1       0.08      0.84      0.14        94

avg / total       1.00      0.98      0.99     56962



In [10]:
    clf.best_params_

{'class_weight': 'balanced',
 'criterion': 'gini',
 'max_depth': 7,
 'n_estimators': 32}

# Iteration 3: Near miss version 2

In [11]:
nm = NearMiss(ratio = 'majority', random_state=42, version = 2)
X_res, y_res = nm.fit_sample(X_train, y_train)

clf_base = RandomForestClassifier()
parameters = {'n_estimators': [16],
             'criterion': ['gini'],
             'max_depth': [5]}
k_fold = KFold(n_splits = 10, shuffle = True, random_state = 42)
clf = GridSearchCV(clf_base, param_grid = parameters, cv = k_fold, n_jobs = 3, scoring = 'f1_macro')
clf.fit(X_res, y_res)
prediction = clf.predict(X_test)

In [12]:
print (classification_report(y_test, prediction))

             precision    recall  f1-score   support

          0       1.00      0.59      0.74     56868
          1       0.00      0.94      0.01        94

avg / total       1.00      0.59      0.74     56962



# Iteration 4: Tomek Links

In [13]:
tl = TomekLinks(ratio = 'majority', random_state = 42)
X_res, y_res = tl.fit_sample(X_train, y_train)

clf_base = RandomForestClassifier(random_state = 42)
parameters = {'n_estimators': [64],
             'criterion': ['gini'],
             'max_depth': [10],
             'min_samples_split': [3],
             'max_features': ['auto']}
k_fold = KFold(n_splits = 10, shuffle = True, random_state = 42)
clf = GridSearchCV(clf_base, param_grid = parameters, cv = k_fold, n_jobs = 3, scoring = 'f1_macro')
clf.fit(X_res, y_res)
prediction = clf.predict(X_test)

In [14]:
print (classification_report(y_test, prediction))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00     56868
          1       0.97      0.70      0.81        94

avg / total       1.00      1.00      1.00     56962



In [15]:
print (confusion_matrix(y_test, prediction))

[[56866     2]
 [   28    66]]


In [16]:
clf.best_params_

{'criterion': 'gini',
 'max_depth': 10,
 'max_features': 'auto',
 'min_samples_split': 3,
 'n_estimators': 64}

# Iteration 5: SMOTE

In [17]:
smote = SMOTE(ratio = 'minority', random_state = 42)
X_res, y_res = smote.fit_sample(X_train, y_train)

clf_base = RandomForestClassifier(random_state = 42)
parameters = {'n_estimators': [16],
             'criterion': ['gini'],
             'max_depth': [7],
             'class_weight': [None]}
k_fold = KFold(n_splits = 10, shuffle = True, random_state = 42)
clf = GridSearchCV(clf_base, param_grid = parameters, cv = k_fold, n_jobs = 3, scoring = 'f1_macro')
clf.fit(X_res, y_res)
prediction = clf.predict(X_test)

In [18]:
print (classification_report(y_test, prediction))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00     56868
          1       0.32      0.81      0.46        94

avg / total       1.00      1.00      1.00     56962



In [19]:
clf.best_params_

{'class_weight': None, 'criterion': 'gini', 'max_depth': 7, 'n_estimators': 16}

# Iteration 6: SMOTE + Tomek Links

In [20]:
smote = SMOTE(ratio = 'minority', random_state = 42)
X_smote, y_smote = smote.fit_sample(X_train, y_train)
tl = TomekLinks(ratio = 'majority', random_state = 42)
X_res, y_res = tl.fit_sample(X_smote, y_smote)

clf_base = RandomForestClassifier(random_state = 42)
parameters = {'n_estimators': [8],
             'criterion': ['gini'],
             'max_depth': [10],
             'class_weight': [None]}
k_fold = KFold(n_splits = 10, shuffle = True, random_state = 42)
clf = GridSearchCV(clf_base, param_grid = parameters, cv = k_fold, n_jobs = 3, scoring = 'f1_macro')
clf.fit(X_res, y_res)
prediction = clf.predict(X_test)

In [21]:
print (classification_report(y_test, prediction))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00     56868
          1       0.40      0.79      0.53        94

avg / total       1.00      1.00      1.00     56962



# Iteration 7: Repeated edited nearest neighbours

In [22]:
r_enn = RepeatedEditedNearestNeighbours(ratio = 'majority', random_state = 42)
X_res, y_res = r_enn.fit_sample(X_train, y_train)

clf_base = RandomForestClassifier(random_state = 42)
parameters = {'n_estimators': [8],
             'criterion': ['gini'],
             'max_depth': [9, 10],
             'min_samples_split': [3, 4]}
k_fold = KFold(n_splits = 10, shuffle = True, random_state = 42)
clf = GridSearchCV(clf_base, param_grid = parameters, cv = k_fold, n_jobs = 3, scoring = 'f1_macro')
clf.fit(X_res, y_res)
prediction = clf.predict(X_test)

In [23]:
print (classification_report(y_test, prediction))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00     56868
          1       0.93      0.69      0.79        94

avg / total       1.00      1.00      1.00     56962



clf.best_params_

# Iteration 8: ADASYN

In [24]:
adasyn = ADASYN(ratio = 'minority', random_state = 42)
X_res, y_res = adasyn.fit_sample(X_train, y_train)

clf_base = RandomForestClassifier(random_state = 42)
parameters = {'n_estimators': [16],
             'criterion': ['gini'],
             'max_depth': [7],
             'class_weight': [None]}
k_fold = KFold(n_splits = 10, shuffle = True, random_state = 42)
clf = GridSearchCV(clf_base, param_grid = parameters, cv = k_fold, n_jobs = 3, scoring = 'f1_macro')
clf.fit(X_res, y_res)
prediction = clf.predict(X_test)

In [25]:
print (classification_report(y_test, prediction))

             precision    recall  f1-score   support

          0       1.00      0.98      0.99     56868
          1       0.07      0.87      0.13        94

avg / total       1.00      0.98      0.99     56962



# Iteration 9: SMOTE + RENN

In [26]:
smote = SMOTE(ratio = 'minority', random_state = 42)
X_smote, y_smote = smote.fit_sample(X_train, y_train)
r_enn = RepeatedEditedNearestNeighbours(ratio = 'majority', random_state = 42)
X_res, y_res = r_enn.fit_sample(X_train, y_train)

clf_base = RandomForestClassifier(random_state = 42)
parameters = {'n_estimators': [64],
             'criterion': ['gini'],
             'max_depth': [10],
             'min_samples_split': [3]}
k_fold = KFold(n_splits = 10, shuffle = True, random_state = 42)
clf = GridSearchCV(clf_base, param_grid = parameters, cv = k_fold, n_jobs = 3, scoring = 'f1_macro')
clf.fit(X_res, y_res)
prediction = clf.predict(X_test)

In [27]:
print (classification_report(y_test, prediction))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00     56868
          1       0.96      0.72      0.82        94

avg / total       1.00      1.00      1.00     56962



In [28]:
print (confusion_matrix(y_test, prediction))

[[56865     3]
 [   26    68]]
